In [4]:
import os
import zipfile
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.models import resnet50
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torch_geometric.nn import GCNConv
import pandas as pd
from PIL import Image


In [8]:
!pip install tensorflow


   ---------------------------------------- 0.0/390.3 MB ? eta -:--:--
   ---------------------------------------- 2.1/390.3 MB 9.0 MB/s eta 0:00:43
   ---------------------------------------- 4.5/390.3 MB 9.9 MB/s eta 0:00:39
    --------------------------------------- 6.6/390.3 MB 10.3 MB/s eta 0:00:38
    --------------------------------------- 8.7/390.3 MB 10.3 MB/s eta 0:00:37
   - -------------------------------------- 11.8/390.3 MB 11.2 MB/s eta 0:00:34
   - -------------------------------------- 14.7/390.3 MB 11.7 MB/s eta 0:00:33
   - -------------------------------------- 17.6/390.3 MB 11.9 MB/s eta 0:00:32
   -- ------------------------------------- 21.0/390.3 MB 12.5 MB/s eta 0:00:30
   -- ------------------------------------- 24.1/390.3 MB 12.9 MB/s eta 0:00:29
   -- ------------------------------------- 27.3/390.3 MB 13.3 MB/s eta 0:00:28
   -- ------------------------------------- 27.3/390.3 MB 13.3 MB/s eta 0:00:28
   -- ------------------------------------- 27.3/390.3 

In [13]:
!pip install spektral

In [10]:
import os
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.neighbors import kneighbors_graph
import pandas as pd
from spektral.layers import GCNConv
from spektral.data import Graph

In [38]:
def custom_data_generator(data_dir, labels_csv, img_size=(224, 224), batch_size=32, validation_split=0.2):
    labels_df = pd.read_csv(labels_csv)
    labels_df['file_path'] = labels_df['id_code'].apply(lambda x: os.path.join(data_dir, f"{x}.png"))
    labels_df['diagnosis'] = labels_df['diagnosis'].astype(str)  # Convert labels to strings

    train_df = labels_df.sample(frac=1 - validation_split, random_state=42)
    val_df = labels_df.drop(train_df.index)

    datagen = ImageDataGenerator(rescale=1.0 / 255)
    train_gen = datagen.flow_from_dataframe(
        train_df, x_col='file_path', y_col='diagnosis',
        target_size=img_size, batch_size=batch_size, class_mode='categorical'
    )
    val_gen = datagen.flow_from_dataframe(
        val_df, x_col='file_path', y_col='diagnosis',
        target_size=img_size, batch_size=batch_size, class_mode='categorical'
    )
    return train_gen, val_gen


In [ ]:
def build_resnet_model(input_shape=(224, 224, 3), num_classes=5):
    base_model = ResNet50(weights='imagenet', include_top=False, input_tensor=Input(shape=input_shape))
    x = GlobalAveragePooling2D()(base_model.output)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    # Freeze the base model layers initially
    for layer in base_model.layers:
        layer.trainable = False
    return model


In [30]:
def build_gcn_model(num_features, num_classes):
    input_node_features = Input(shape=(num_features,))
    input_adjacency = Input(shape=(None,))
    x = GCNConv(32, activation='relu')([input_node_features, input_adjacency])
    x = GCNConv(num_classes, activation='softmax')([x, input_adjacency])
    model = Model(inputs=[input_node_features, input_adjacency], outputs=x)
    return model

In [32]:
# Compute Adjacency Matrix
def compute_adjacency_matrix(features, k=5):
    adjacency_matrix = kneighbors_graph(features, k, mode='connectivity', include_self=True).toarray()
    return adjacency_matrix

In [34]:
def train_hybrid_model(train_gen, val_gen):
    # ResNet Training
    resnet_model = build_resnet_model()
    resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    resnet_model.fit(train_gen, validation_data=val_gen, epochs=5, steps_per_epoch=len(train_gen), validation_steps=len(val_gen))

    # Extract Features
    features = resnet_model.predict(train_gen, verbose=1)
    adjacency_matrix = compute_adjacency_matrix(features)
    
    # GCN Training
    gcn_model = build_gcn_model(features.shape[1], num_classes=5)
    gcn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    graph = Graph(x=features, a=adjacency_matrix, y=train_gen.classes)
    gcn_model.fit([graph.x, graph.a], tf.keras.utils.to_categorical(graph.y, num_classes=5), epochs=10, batch_size=32)
    return resnet_model, gcn_model

In [ ]:
# Run Pipeline
data_directory = 'datasets/APTOS/train_images'
labels_csv = 'datasets/APTOS/train.csv'

train_gen, val_gen = custom_data_generator(data_directory, labels_csv)
resnet_model, gcn_model = train_hybrid_model(train_gen, val_gen)


Found 2930 validated image filenames belonging to 5 classes.
Found 732 validated image filenames belonging to 5 classes.


C:\Users\netcom\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/5


C:\Users\netcom\anaconda3\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 224, 224, 3))
  warnings.warn(msg)


92/92 ━━━━━━━━━━━━━━━━━━━━ 403s 4s/step - accuracy: 0.3513 - loss: 1.6420 - val_accuracy: 0.5000 - val_loss: 1.2997
Epoch 2/5
12/92 ━━━━━━━━━━━━━━━━━━━━ 4:20 3s/step - accuracy: 0.4675 - loss: 1.3178